In [8]:
import glob
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

In [2]:
out_file = "/sensei-fs/users/chuongh/InstMatt/datasets/VIM_train"
data_root = "/mnt/localssd/syn/train"

In [3]:
all_fgrs = []
all_alphas = []

video_names = glob.glob(os.path.join(data_root, "fgr") + "/*")
video_names.sort()

for video_name in video_names:
    framenames = glob.glob(os.path.join(video_name, "*.jpg"))
    framenames.sort()
    alpha_names = glob.glob(video_name.replace("fgr", "pha") + "/*/*.png")
    alpha_names.sort()
    n_i = len(alpha_names) // len(framenames)
    for j in range(n_i):
        all_fgrs.extend(framenames)
        all_alphas.extend([x.replace("fgr", "pha").replace(".jpg", "") + f"/{j:02d}.png" for x in framenames])

In [10]:
with open(out_file + "_fg.txt", "w") as f:
    f.write("\n".join(all_fgrs))

In [11]:
with open(out_file + "_alpha.txt", "w") as f:
    f.write("\n".join(all_alphas))

In [9]:
# Extract fg, bg based on frame and alpha

for fg_path, alpha_path in tqdm(zip(all_fgrs, all_alphas)):
    _f = cv2.imread(fg_path, cv2.IMREAD_UNCHANGED)
    _a = cv2.imread(alpha_path, cv2.IMREAD_GRAYSCALE)
    _a[_a <= 1] = 0

    # Generate FG and BG by remove color by alpha
    _bg = _f.copy()
    _af = _a.astype(np.float32) / 255.
    _af = _af[..., np.newaxis]
    _af = np.repeat(_af, 3, axis=2)
    fg_region = (_f[_af > 0].astype(np.float32) / _af[_af > 0]).astype(np.uint8)
    fg_region = np.clip(fg_region, 0, 255)
    bg_region = (_f[_af > 0].astype(np.float32) - fg_region * _af[_af >0]) / (1. - _af[_af > 0])
    np.nan_to_num(bg_region, copy=False)
    bg_region = np.clip(bg_region, 0, 255)
    bg_region = bg_region.astype(np.uint8)
    _f[_af > 0] = fg_region
    _bg[_af > 0] = bg_region
    
    fgr = Image.fromarray(_f[:, :, ::-1])
    bgr = Image.fromarray(_bg[:, :, ::-1])
    
    out_fg_path = fg_path.replace("/fgr/", "/fgr_extracted/")
    out_bg_path = fg_path.replace("/fgr/", "/bgr_extracted/")
    os.makedirs(os.path.dirname(out_fg_path), exist_ok=True)
    os.makedirs(os.path.dirname(out_bg_path), exist_ok=True)
    fgr.save(out_fg_path)
    bgr.save(out_bg_path)

0it [00:00, ?it/s]/tmp/ipykernel_3414852/1933025538.py:15: RuntimeWarning: invalid value encountered in divide
  bg_region = (_f[_af > 0].astype(np.float32) - fg_region * _af[_af >0]) / (1. - _af[_af > 0])
988it [03:25,  4.81it/s]


KeyboardInterrupt: 

Bad pipe message: %s [b'\x0em\xd3xM\xc8\xfdpi\xa5/F\xe7\x92\t\xff\xd0\\ \x18\xb2g\x15\x19\xae\n\xe44\xe2[\x96\x92n\xab \x96\xed\xf19H\xbfND\xc6p\xd8\xf9\xccj\x88?\x00>\x13\x02\x13\x03\x13\x01\xc0,\xc00\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0+\xc0/\x00\x9e\xc0$\xc0(']
